In [8]:
import numpy as np
from quri_parts.circuit import H, CZ
from sklearn.datasets import load_iris, load_digits
from sklearn.preprocessing import MinMaxScaler
from circuit import LearningCircuit
from numpy.typing import NDArray
from quri_parts.qulacs.overlap_estimator import create_qulacs_vector_overlap_estimator
from quri_parts.core.state import quantum_state, GeneralCircuitQuantumState


def create_quantum_circuit():
    qc = LearningCircuit(n_qubits)

    def preprocess_x(x: NDArray[np.float64], index: int) -> float:
        xa = x[index % len(x)]
        return min(1, max(-1, xa))

    for i in range(n_qubits):
        qc.add_gate(H(i))
    for d in range(depth):
        for i in range(n_qubits):
            qc.add_input_RY_gate(i, lambda x, i=i: preprocess_x(x, i))
        for i in range(n_qubits):
            qc.add_input_RX_gate(i, lambda x, i=i: preprocess_x(x, i))
        if d < depth - 1:
            for i in range(n_qubits):
                qc.add_gate(CZ(i, (i + 1) % n_qubits))
    return qc


X_train, y_train = load_iris(return_X_y=True)
# X_train = X_train / 16.
# X_train = X_train[:200]
# y_train = y_train[:200] #500 => fidelity 81.6[s]
n_qubits = 12
depth = 1
scaler = MinMaxScaler((0, np.pi / 2))
n_qubits = 12
depth = 1

X_train = scaler.fit_transform(X_train)

pqc = create_quantum_circuit()
x_0 = pqc.bind_input_and_parameters(X_train[0], [])
x_1 = pqc.bind_input_and_parameters(X_train[15], [])
estimator = create_qulacs_vector_overlap_estimator()
print(
    estimator(
        quantum_state(n_qubits=n_qubits, circuit=x_0), quantum_state(n_qubits=n_qubits, circuit=x_1)
    )
)

_Estimate(value=0.9270404554320588, error=0.0)


In [10]:
sq_distance = np.zeros((len(X_train), len(X_train)))
for i in range(len(X_train)):
    for j in range(i + 1, len(X_train)):
        inner_prod = estimator(
            quantum_state(n_qubits=n_qubits, circuit=pqc.bind_input_and_parameters(X_train[i], [])),
            quantum_state(n_qubits=n_qubits, circuit=pqc.bind_input_and_parameters(X_train[j], [])),
        )
        sq_distance[i][j] = 1 - inner_prod[0].real
        sq_distance[j][i] = sq_distance[i][j]
print(sq_distance)

[[0.         0.12065636 0.06844121 ... 0.84985485 0.80410116 0.80592477]
 [0.12065636 0.         0.02461169 ... 0.84485567 0.83054096 0.7926377 ]
 [0.06844121 0.02461169 0.         ... 0.86451119 0.83848435 0.81397022]
 ...
 [0.84985485 0.84485567 0.86451119 ... 0.         0.09282863 0.0760118 ]
 [0.80410116 0.83054096 0.83848435 ... 0.09282863 0.         0.09130359]
 [0.80592477 0.7926377  0.81397022 ... 0.0760118  0.09130359 0.        ]]
